In [1]:
import os
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [2]:
titlesize = 16
labelsize = 16
legendsize = 16
xticksize = 16
yticksize = xticksize

plt.rcParams['legend.markerscale'] = 1.5  # the relative size of legend markers vs. original
plt.rcParams['legend.handletextpad'] = 0.5
plt.rcParams['legend.labelspacing'] = 0.4  # the vertical scape between the legend entries in 
plt.rcParams['legend.borderpad'] = 0.5  # border whitescape in fontsize units
plt.rcParams['font.size'] = 12
plt.rcParams['font.serif'] = 'Times New Roman'
plt.rcParams['axes.labelsize'] = labelsize
plt.rcParams['axes.titlesize'] = titlesize
plt.rcParams['figure.figsize'] = (10, 6)

plt.rc('xtick', labelsize=xticksize)
plt.rc('ytick', labelsize=yticksize)
plt.rc('legend', fontsize=legendsize)

In [3]:
URL_BASE = 'https://raw.githubusercontent.com/ab-courses/simulator-ab-datasets/main/2022-04-01/'

def read_database(file_name):
    return pd.read_csv(os.path.join(URL_BASE, file_name))

# 1. Популярные товары
Определить топ-3 товара по суммарной выручке

In [91]:
df_sales = read_database('2022-04-01T12_df_sales.csv')
df_sales_details = read_database('2022-04-01T12_df_sales_detail.csv')

df_sales['date'] = pd.to_datetime(df_sales['date'])
df_sales_details['date'] = pd.to_datetime(df_sales_details['date'])

In [94]:
df_sales_details.groupby('good')['price'].sum().sort_values()[-3:].index.tolist()

['chicken bbq pizza', 'double pepperoni pizza', 'chefs pizza']

# 2. Время от захода на сайт до покупки
Средний срок от захода юзера на сайт до совершения покупки.
Заход на сайт относится к покупке, если он был совершен не ранее, чем за два часа до совершения покупки.

Полученный результат перевести в минуты и округлить до целого значения.

In [43]:
df_sales = read_database('2022-04-01T12_df_sales.csv')
df_web_logs = read_database('2022-04-01T12_df_web_logs.csv')

df_sales['date'] = pd.to_datetime(df_sales['date'])
df_web_logs['date'] = pd.to_datetime(df_web_logs['date'])

In [98]:
df_web_logs.head()

,user_id,page,web_date,load_time
0,f25239,m,2022-02-03 23:45:37,80.8
1,06d6df,m,2022-02-03 23:49:56,70.5
2,06d6df,m,2022-02-03 23:51:16,89.7
3,f25239,m,2022-02-03 23:51:43,74.4
4,697870,m,2022-02-03 23:53:12,66.8


In [45]:
df_sales['date_start'] = df_sales['date'] - pd.Timedelta(hours=2)

In [47]:
df_sales = df_sales.rename(columns={'date': 'sale_date'})
df_web_logs = df_web_logs.rename(columns={'date': 'web_date'})

In [48]:
df_sales = df_sales.merge(df_web_logs, how='outer', on ='user_id')

In [49]:
df_sales = df_sales[~df_sales['sale_id'].isna()]

In [50]:
df_sales = df_sales[df_sales['web_date'] >= df_sales['date_start']]

In [51]:
df_sales = df_sales[df_sales['web_date'] <= df_sales['sale_date']]

In [53]:
df_sales['date_diff'] = df_sales['sale_date'] - df_sales['web_date']

In [55]:
df_sales['seconds_diff'] = df_sales['date_diff'].dt.total_seconds()

In [56]:
df_sales_agg = df_sales.groupby(by='sale_id', as_index=False).agg({'user_id' :'max', 
                                                                   'seconds_diff' : 'max', 
                                                                   'sale_date' : 'max', 
                                                                   'web_date' : 'min'
                                                                  })

In [61]:
df_sales_agg['seconds_diff'].mean()/60

16.58854451295988

# Какая доля пользователей, совершивших покупку в феврале, совершила покупку и в марте?

Ответ округлите с точностью до 2-го знака после точки.

In [67]:
df_sales = read_database('2022-04-01T12_df_sales.csv')

df_sales['date'] = pd.to_datetime(df_sales['date'])
# df_sales['day'] = df_sales['date'].dt.date
df_sales['month'] = df_sales['date'].dt.month
df_sales['year'] = df_sales['date'].dt.year

In [97]:
df_sales.head()

,sale_id,date,count_pizza,count_drink,price,user_id
0,1000001,2022-02-04 10:00:24,1,0,720,1c1543
1,1000002,2022-02-04 10:02:28,1,1,930,a9a6e8
2,1000003,2022-02-04 10:02:35,3,1,1980,23420a
3,1000004,2022-02-04 10:03:06,1,1,750,3e8ed5
4,1000005,2022-02-04 10:03:23,1,1,870,cbc468


In [71]:
df_sales_feb = df_sales[df_sales['month'] == 2][['user_id', 'sale_id']].groupby(by='user_id', 
                                                                                as_index=False).agg({'sale_id': 'nunique'})

In [74]:
df_sales_mar = df_sales[df_sales['month'] == 3][['user_id', 'sale_id']].groupby(by='user_id', 
                                                                                as_index=False).agg({'sale_id': 'nunique'})

In [76]:
len(df_sales_mar.merge(df_sales_feb, how='inner', on='user_id'))/len(df_sales_feb)

0.658493870402802

# Реализация методов для оценки АБ экспериментов

В практических заданиях на программирование нашей целью будет написать код, с помощью которого можно будет оценивать A/B эксперименты. В результате у вас будет готова основа для создания платформы A/B тестирования, похожей на нашу или даже лучше.

Мы разделили функции A/B платформы на 4 блока:
- DataService — предоставляет доступ к сырым данным;
- MetricsService — вычисляет метрики;
- ExperimentsService — оценивает эксперименты;
- SplittingService — подбирает группы пользователей для эксперимента.

# Реализация метода для доступа к сырым данным
Напишите метод get_data_subset класса DataService. Описание метода есть в шаблоне решения ниже.

Скопируйте код шаблона в py-файл, допишите недостающие части, сохраните изменения и отправьте файл в проверяющую систему.

Для самопроверки используйте пример теста решения в нижней части шаблона. При проверке решения будут использоваться другие тестовые данные

In [95]:
import pandas as pd

from datetime import datetime


class DataService:

    def __init__(self, table_name_2_table):
        """Класс, предоставляющий доступ к сырым данным.
        
        :param table_name_2_table (dict[str, pd.DataFrame]): словарь таблиц с данными.
            Пример, {
                'sales': pd.DataFrame({'sale_id': ['123', ...], ...}),
                ...
            }. 
        """
        self.table_name_2_table = table_name_2_table

    def get_data_subset(self, table_name, begin_date, end_date, user_ids=None, columns=None):
        """Возвращает подмножество данных.

        :param table_name (str): название таблицы с данными.
        :param begin_date (datetime.datetime): дата начала интервала с данными.
            Пример, df[df['date'] >= begin_date].
            Если None, то фильтровать не нужно.
        :param end_date (None, datetime.datetime): дата окончания интервала с данными.
            Пример, df[df['date'] < end_date].
            Если None, то фильтровать не нужно.
        :param user_ids (None, list[str]): список user_id, по которым нужно предоставить данные.
            Пример, df[df['user_id'].isin(user_ids)].
            Если None, то фильтровать по user_id не нужно.
        :param columns (None, list[str]): список названий столбцов, по которым нужно предоставить данные.
            Пример, df[columns].
            Если None, то фильтровать по columns не нужно.

        :return df (pd.DataFrame): датафрейм с подмножеством данных.
        """
        # YOUR_CODE_HERE
        df = self.table_name_2_table[table_name]
        if begin_date is not None:
            df = df[df['date'] >= begin_date]
        if end_date is not None:
            df = df[df['date'] < end_date]
        if user_ids is not None:
            df = df[df['user_id'].isin(user_ids)]
        if columns is not None:
            df = df[columns]
        return df.copy()


def _chech_df(df, df_ideal, sort_by):
    assert isinstance(df, pd.DataFrame), 'Функция вернула не pd.DataFrame.'
    assert len(df) == len(df_ideal), 'Неверное количество строк.'
    assert len(df.T) == len(df_ideal.T), 'Неверное количество столбцов.'
    columns = df_ideal.columns
    assert df.columns.isin(columns).sum() == len(df.columns), 'Неверное название столбцов.'
    df = df[columns].sort_values(sort_by)
    df_ideal = df_ideal.sort_values(sort_by)
    assert df_ideal.equals(df), 'Итоговый датафрейм не совпадает с верным результатом.'

In [96]:
if __name__ == '__main__':
    table = pd.DataFrame({
        'date': [datetime(2022, 1, 5, 12,), datetime(2022, 1, 7, 12)],
        'user_id': ['1', '2'],
    })
    ideal_df = pd.DataFrame({
        'date': [datetime(2022, 1, 5, 12,)],
        'user_id': ['1'],
    })

    data_service = DataService({'table': table})
    res_df = data_service.get_data_subset('table', datetime(2022, 1, 1), datetime(2022, 1, 6))
    _chech_df(res_df, ideal_df, 'date')
    print('simple test passed')

simple test passed


# Реализация методов для вычисления метрик
Напишите методы для вычисления метрик «revenue (web)», «revenue (all)», «response time». Описание методов есть в шаблоне решения ниже.

Скопируйте код шаблона в py-файл, допишите недостающие части, сохраните изменения и отправьте файл в проверяющую систему.

Для самопроверки внизу шаблона есть пример теста решения. При проверке решения будут использоваться другие тестовые данные.

In [125]:
import pandas as pd

from datetime import datetime


class DataService:

    def __init__(self, table_name_2_table):
        self.table_name_2_table = table_name_2_table

    def get_data_subset(self, table_name, begin_date, end_date, user_ids=None, columns=None):
        df = self.table_name_2_table[table_name]
        if begin_date:
            df = df[df['date'] >= begin_date]
        if end_date:
            df = df[df['date'] < end_date]
        if user_ids:
            df = df[df['user_id'].isin(user_ids)]
        if columns:
            df = df[columns]
        return df.copy()


class MetricsService:

    def __init__(self, data_service):
        """Класс для вычисления метрик.

        :param data_service (DataService): объект класса, предоставляющий доступ к данным.
        """
        self.data_service = data_service

    def _get_data_subset(self, table_name, begin_date, end_date, user_ids=None, columns=None):
        """Возвращает часть таблицы с данными."""
        return self.data_service.get_data_subset(table_name, begin_date, end_date, user_ids, columns)

    def _calculate_response_time(self, begin_date, end_date, user_ids):
        """Вычисляет значения времени обработки запроса сервером.
        
        Нужно вернуть значения user_id и load_time из таблицы 'web-logs', отфильтрованные по date и user_id.
        Считаем, что каждый запрос независим, поэтому группировать по user_id не нужно.

        :param begin_date, end_date (datetime): период времени, за который нужно считать значения.
        :param user_id (None, list[str]): id пользователей, по которым нужно отфильтровать полученные значения.
        
        :return (pd.DataFrame): датафрейм с двумя столбцами ['user_id', 'metric']
        """
        # YOUR_CODE_HERE
        return (
            self._get_data_subset('web-logs', begin_date, end_date, user_ids, ['user_id', 'load_time'])
            .rename(columns={'load_time' : 'metric'})
            [['user_id', 'metric']]
        )
        
    def _calculate_revenue_web(self, begin_date, end_date, user_ids):
        """Вычисляет значения выручки с пользователя за указанный период
        для заходивших на сайт в указанный период.

        Эти данные нужны для экспериментов на сайте, когда в эксперимент попадают только те, кто заходил на сайт.
        
        Нужно вернуть значения user_id и выручки (sum(price)).
        Данные о ценах в таблице 'sales'. Данные о заходивших на сайт в таблице 'web-logs'.
        Если пользователь зашёл на сайт и ничего не купил, его суммарная стоимость покупок равна нулю.
        Для каждого user_id должно быть ровно одно значение.

        :param begin_date, end_date (datetime): период времени, за который нужно считать значения.
            Также за этот период времени нужно выбирать пользователей, которые заходили на сайт.
        :param user_id (None, list[str]): id пользователей, по которым нужно отфильтровать полученные значения.
        
        :return (pd.DataFrame): датафрейм с двумя столбцами ['user_id', 'metric']
        """
        # YOUR_CODE_HERE
        user_ids_ = (
            self._get_data_subset('web-logs', begin_date, end_date, user_ids, ['user_id'])['user_id'].unique()
        )
        
        df = (
            self._get_data_subset('sales', begin_date, end_date, user_ids, ['user_id', 'price'])
            .groupby(by='user_id')[['price']].sum().reset_index()
            .rename(columns={'price' : 'metric'})
        )
        df = pd.merge(pd.DataFrame({'user_id' : user_ids_}), df, how='left', on='user_id').fillna(0)
        
        return df[['user_id', 'metric']]

    def _calculate_revenue_all(self, begin_date, end_date, user_ids):
        """Вычисляет значения выручки с пользователя за указанный период
        для заходивших на сайт до end_date.

        Эти данные нужны, например, для экспериментов с рассылкой по email,
        когда в эксперимент попадают те, кто когда-либо оставил нам свои данные.
        
        Нужно вернуть значения user_id и выручки (sum(price)).
        Данные о ценах в таблице 'sales'. Данные о заходивших на сайт в таблице 'web-logs'.
        Если пользователь ничего не купил за указанный период, его суммарная стоимость покупок равна нулю.
        Для каждого user_id должно быть ровно одно значение.

        :param begin_date, end_date (datetime): период времени, за который нужно считать значения.
            Нужно выбирать пользователей, которые хотя бы раз заходили на сайт до end_date.
        :param user_id (None, list[str]): id пользователей, по которым нужно отфильтровать полученные значения.
        
        :return (pd.DataFrame): датафрейм с двумя столбцами ['user_id', 'metric']
        """
        # YOUR_CODE_HERE
        user_ids_ = (
            self._get_data_subset('web-logs', None, end_date, user_ids, ['user_id'])
            ['user_id'].unique()
        )
        df = (
            self._get_data_subset('sales', begin_date, end_date, user_ids, ['user_id', 'price'])
            .groupby(by='user_id')[['price']].sum().reset_index()
            .rename(columns={'price' : 'metric'})
        )
        df = pd.merge(pd.DataFrame({'user_id' : user_ids_}), df, how='left', on='user_id').fillna(0)
        
        return df[['user_id', 'metric']]

    def calculate_metric(self, metric_name, begin_date, end_date, user_ids=None):
        """Считает значения для вычисления метрик.

        :param metric_name (str): название метрики
        :param begin_date (datetime): дата начала периода (включая границу)
        :param end_date (datetime): дата окончания периода (не включая границу)
        :param user_ids (list[str], None): список пользователей.
            Если None, то вычисляет значения для всех пользователей.
        :return df: columns=['user_id', 'metric']
        """
        if metric_name == 'response time':
            return self._calculate_response_time(begin_date, end_date, user_ids)
        elif metric_name == 'revenue (web)':
            return self._calculate_revenue_web(begin_date, end_date, user_ids)
        elif metric_name == 'revenue (all)':
            return self._calculate_revenue_all(begin_date, end_date, user_ids)
        else:
            raise ValueError('Wrong metric name')


def _chech_df(df, df_ideal, sort_by, reindex=False, set_dtypes=False):
    assert isinstance(df, pd.DataFrame), 'Функция вернула не pd.DataFrame.'
    assert len(df) == len(df_ideal), 'Неверное количество строк.'
    assert len(df.T) == len(df_ideal.T), 'Неверное количество столбцов.'
    columns = df_ideal.columns
    assert df.columns.isin(columns).sum() == len(df.columns), 'Неверное название столбцов.'
    df = df[columns].sort_values(sort_by)
    df_ideal = df_ideal.sort_values(sort_by)
    if reindex:
        df_ideal.index = range(len(df_ideal))
        df.index = range(len(df))
    if set_dtypes:
        for column, dtype in df_ideal.dtypes.to_dict().items():
            df[column] = df[column].astype(dtype)
    assert df_ideal.equals(df), 'Итоговый датафрейм не совпадает с верным результатом.'


In [126]:
if __name__ == '__main__':
    df_sales = pd.DataFrame({
        'sale_id': [1, 2, 3],
        'date': [datetime(2022, 3, day, 11) for day in range(11, 14)],
        'price': [1100, 900, 1500],
        'user_id': ['1', '2', '1'],
    })
    df_web_logs = pd.DataFrame({
        'date': [datetime(2022, 3, day, 11) for day in range(10, 14)],
        'load_time': [80.8, 90.1, 15.8, 19.7],
        'user_id': ['3', '1', '2', '1'],
    })
    begin_date = datetime(2022, 3, 11, 9)
    end_date = datetime(2022, 4, 11, 9)

    ideal_response_time = pd.DataFrame({'user_id': ['1', '2', '1'], 'metric': [90.1, 15.8, 19.7],})
    ideal_revenue_web = pd.DataFrame({'user_id': ['1', '2'], 'metric': [2600., 900.],})
    ideal_revenue_all = pd.DataFrame({'user_id': ['1', '2', '3'], 'metric': [2600., 900., 0.],})

    data_service = DataService({'sales': df_sales, 'web-logs': df_web_logs})
    metrics_service = MetricsService(data_service)

    df_response_time = metrics_service.calculate_metric('response time', begin_date, end_date)
    df_revenue_web = metrics_service.calculate_metric('revenue (web)', begin_date, end_date)
    df_revenue_all = metrics_service.calculate_metric('revenue (all)', begin_date, end_date)

    _chech_df(df_response_time, ideal_response_time, ['user_id', 'metric'], True, True)
    _chech_df(df_revenue_web, ideal_revenue_web, ['user_id', 'metric'], True, True)
    _chech_df(df_revenue_all, ideal_revenue_all, ['user_id', 'metric'], True, True)
    print('simple test passed')


simple test passed
